## **Import Necessary Dependencies**

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv

from pinecone import Pinecone, ServerlessSpec, init
import pinecone
#from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import VertexAIEmbeddings
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from datasets import load_dataset
from langchain_community.retrievers import PineconeHybridSearchRetriever
from tqdm.autonotebook import tqdm

import warnings
warnings.filterwarnings('ignore')
print("Installation Complete.")

e:\Git Uploads\RAGtime\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Installation Complete.


In [3]:
# Load the 'ELI5' dataset from Huggingface
dataset = load_dataset("squad_v2")

print(dataset['train'][0])
print(dataset)

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
      

**Pre-Process Data**

In [4]:
def preprocess_squad(dataset):
    formatted_data = []

    for split in ["train", "validation"]:
        for row in dataset[split]:
            q, context = row["question"], row["context"]
            for a in row["answers"]["text"]:
                formatted_data.append({"question": q, "answer": a, 'context': context})

    df = pd.DataFrame(formatted_data)

    # Removing newlines
    df["question"] = df["question"].str.replace("\n", " ", regex=False)
    df["answer"] = df["answer"].str.replace("\n", " ", regex=False)
    df["context"] = df["context"].str.replace("\n", " ", regex=False)

    return df

In [5]:
df = preprocess_squad(dataset)

print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| question                                                         | answer              | context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|:-----------------------------------------------------------------|:--------------------|:-------------------------------------------------------------------------------------------------------------------------

In [6]:
load_dotenv()

#open_api_key = os.environ.get('OPEN_API_KEY')
api_key = os.environ.get("PINECONE_API_KEY")
environment = os.environ.get('PINECONE_ENV')

## Initialize the pinecone client
pc = pinecone.Pinecone(api_key=api_key, environment=environment)

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region =  os.environ.get('PINECONE_ENV') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "ragtime"

index = pinecone.Index(index_name, host="https://ragtime-luwubl1.svc.aped-4627-b74a.pinecone.io")

**Embedding Data**

In [7]:
import json

with open("F:\Downloads\inspired-studio-431021-m1-6a0e90d02e4d.json", "r") as f:
    credentials = json.load(f)


In [10]:
#from google.oauth2 import service_account

cred_path = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

#credentials = service_account.Credentials.from_service_account_file(cred_path)

embeddings = VertexAIEmbeddings(
    model_name = "textembedding-gecko",
    project = "inspired-studio-431021-m1",
    location = "us-central1",
    credentials_path = cred_path
)

batch_size = 32

for i in range(0, len(df), batch_size):
    i_end = min(i + batch_size, len(df))
    
    batch_metadata = [
        {
            'question': raw['question'],
            'answer': raw['answer'],
            'context': raw['context']
        } for _,raw in df.iloc[i:i_end].iterrows()
    ]
    # Get embeddings for the batch
    batch_embeddings = embeddings.embed_documents(df['context'].iloc[i:i_end].tolist())
    
    # Create list of (id, vector, metadata)
    to_upsert = list(zip(df.index[i:i_end].astype(str), batch_embeddings,batch_metadata))
    
    # Upsert to pinecone
    index.upsert(vectors=to_upsert)
    

ConfigError: field "client" not yet prepared so type is still a ForwardRef, you might need to call VertexAIEmbeddings.update_forward_refs().

TypeError: Index.__init__() missing 1 required positional argument: 'host'